# Pinecone

Pinecone은 고성능 벡터 데이터베이스로, AI 및 머신러닝 애플리케이션을 위한 효율적인 벡터 저장 및 검색 솔루션입니다. 

Pinecone, Chroma, Faiss와 같은 벡터 데이터베이스들을 비교해보겠습니다. 

**Pinecone의 장점**

1. 확장성: 대규모 데이터셋에 대해 뛰어난 확장성을 제공합니다.
   
2. 관리 용이성: 완전 관리형 서비스로, 인프라 관리 부담이 적습니다.
   
3. 실시간 업데이트: 데이터의 실시간 삽입, 업데이트, 삭제가 가능합니다.
   
4. 고가용성: 클라우드 기반으로 높은 가용성과 내구성을 제공합니다.
   
5. API 친화적: RESTful/Python API를 통해 쉽게 통합할 수 있습니다.

**Pinecone의 단점**

1. 비용: Chroma나 Faiss에 비해 상대적으로 비용이 높을 수 있습니다.
   
2. 커스터마이징 제한: 완전 관리형 서비스이기 때문에 세부적인 커스터마이징에 제한이 있을 수 있습니다.
   
3. 데이터 위치: 클라우드에 데이터를 저장해야 하므로, 데이터 주권 문제가 있을 수 있습니다.

Chroma나 Faiss와 비교했을 때:

- Chroma/FAISS 오픈소스이며 로컬에서 실행 가능하여 초기 비용이 낮고 데이터 제어가 용이합니다. 커스터마이징의 자유도가 높습니다. 하지만 대규모 확장성 면에서는 Pinecone에 비해 제한적일 수 있습니다.

선택은 프로젝트의 규모, 요구사항, 예산 등을 고려하여 결정해야 합니다. 대규모 프로덕션 환경에서는 Pinecone이 유리할 수 있지만, 소규모 프로젝트나 실험적인 환경에서는 Chroma나 Faiss가 더 적합할 수 있습니다.

**참고**

- [Pinecone 공식 홈페이지](https://docs.pinecone.io/integrations/langchain)
- [Pinecone 랭체인](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)

In [57]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [58]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -U langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH10-VectorStores")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH10-VectorStores


## 업데이트 안내

아래의 기능은 커스텀 구현한 내용이므로 아래의 라이브러리를 반드시 업데이트 후 진행해야 합니다.

In [60]:
# 업데이트 명령어
# !pip install -U langchain-teddynote

## 한글 처리를 위한 불용어 사전

한글 불용어 사전 가져오기 (추후 토크나이저에 사용)

In [62]:
from langchain_teddynote.korean import stopwords

# 한글 불용어 사전 불러오기 (불용어 사전 출처: https://www.ranks.nl/stopwords/korean)
stopwords()[:20]

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다']

## 데이터 전처리

아래는 일반 문서의 전처리 과정입니다. `ROOT_DIR` 하위에 있는 모든 `.pdf` 파일을 읽어와 `document_lsit` 에 저장합니다.

In [71]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

split_docs = []

# 텍스트 파일을 load -> List[Document] 형태로 변환
files = sorted(glob.glob("data/*.pdf"))

for file in files:
    loader = PyMuPDFLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

# 문서 개수 확인
len(split_docs)

867

In [64]:
split_docs[0].page_content

'2024년 1월호'

In [65]:
split_docs[0].metadata

{'source': 'data/SPRi AI Brief_2024년1월호_F.pdf',
 'file_path': 'data/SPRi AI Brief_2024년1월호_F.pdf',
 'page': 0,
 'total_pages': 22,
 'format': 'PDF 1.4',
 'title': 'EU AI 법안에 관한 보고서 초안 주요 내용과 시사점',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hancom PDF 1.3.0.480',
 'producer': 'Hancom PDF 1.3.0.480',
 'creationDate': "D:20240108125247+09'00'",
 'modDate': "D:20240108125247+09'00'",
 'trapped': ''}

Pinecone 에 DB 저장하기 위한 문서 전처리를 수행합니다. 이 과정에서 `metadata_keys` 를 지정할 수 있습니다.

추가로 metadata 를 태깅하고 싶은 경우 사전 처리 작업에서 미리 metadata 를 추가한 뒤 진행합니다.

- `split_docs`: 문서 분할 결과를 담은 List[Document] 입니다.
- `metadata_keys`: 문서에 추가할 metadata 키를 담은 List 입니다.
- `min_length`: 문서의 최소 길이를 지정합니다. 이 길이보다 짧은 문서는 제외합니다.
- `use_basename`: 소스 경로를 기준으로 파일명을 사용할지 여부를 지정합니다. 기본값은 `False` 입니다.

In [66]:
# metadata 를 확인합니다.
split_docs[0].metadata

{'source': 'data/SPRi AI Brief_2024년1월호_F.pdf',
 'file_path': 'data/SPRi AI Brief_2024년1월호_F.pdf',
 'page': 0,
 'total_pages': 22,
 'format': 'PDF 1.4',
 'title': 'EU AI 법안에 관한 보고서 초안 주요 내용과 시사점',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hancom PDF 1.3.0.480',
 'producer': 'Hancom PDF 1.3.0.480',
 'creationDate': "D:20240108125247+09'00'",
 'modDate': "D:20240108125247+09'00'",
 'trapped': ''}

### 문서의 전처리

- 필요한 `metadata` 정보를 추출합니다.
- 최소 길이 이상의 데이만 필터링 합니다.
  
- 문서의 `basename` 을 사용할지 여부를 지정합니다. 기본값은 `False` 입니다.
  - 여기서 `basename` 이란 파일 경로의 가장 마지막 부분을 의미합니다. 
  - 예를 들어, `/Users/teddy/data/document.pdf` 의 경우 `document.pdf` 가 됩니다.

In [77]:
split_docs[0].metadata

{'source': 'data/SPRi AI Brief_2024년1월호_F.pdf',
 'file_path': 'data/SPRi AI Brief_2024년1월호_F.pdf',
 'page': 0,
 'total_pages': 22,
 'format': 'PDF 1.4',
 'title': 'EU AI 법안에 관한 보고서 초안 주요 내용과 시사점',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hancom PDF 1.3.0.480',
 'producer': 'Hancom PDF 1.3.0.480',
 'creationDate': "D:20240108125247+09'00'",
 'modDate': "D:20240108125247+09'00'",
 'trapped': ''}

In [82]:
from langchain_teddynote.community.pinecone import preprocess_documents

contents, metadatas = preprocess_documents(
    split_docs=split_docs,
    metadata_keys=["source", "page", "author"],
    min_length=5,
    use_basename=True,
)

  0%|          | 0/867 [00:00<?, ?it/s]

In [86]:
# VectorStore 에 저장할 문서 확인
contents[:5]

['2024년 1월호\nⅠ. 인공지능 산업 동향 브리프\n 1. 정책/법제 \n   ▹ 유럽연합, 세계 최초의 AI 규제 법안에 잠정 합의····························································  1\n   ▹ 베이징 인터넷법원, AI 생성 이미지의 무단 사용에 저작권법 위반 판결·························  2\n \n 2. 기업/산업',
 '2. 기업/산업 \n   ▹ 구글, 멀티모달 AI 모델 ‘제미나이’ 공개············································································  3\n   ▹ 구글 클라우드, 기업용 AI 플랫폼에 이미지 생성 AI ‘이매진 2’ 추가  ·························  4\n   ▹ 앤스로픽, 20만 개의 토큰을 입력할 수 있는 ‘클로드 2.1’ 공개 ···································  5',
 '▹ 스태빌리티AI, 동영상 생성 AI ‘스테이블 비디오 디퓨전’ 공개  ····································  6\n   ▹ 온디바이스 AI, 2024년 주요 기술로 부상할 전망 ··························································  7\n   ▹ 딜로이트, 2024년 생성 AI 반도체 시장 규모 500억 달러 예측 ···································  8',
 '▹ IBM과 메타, 개방형 혁신을 위한 ‘AI 얼라이언스’ 결성···················································  9\n   ▹ 마이크로소프트, 영국 AI 인프라에 2026년까지 32억 달러 투자 계획·························  10\n   ▹ AMD, 생성 AI에 최적화된 ‘인스팅트 MI300’ 시리즈 발표·······

In [87]:
# VectorStore 에 저장할 metadata 확인
metadatas.keys()

dict_keys(['source', 'page', 'author'])

In [88]:
# metadata 에서 source 를 확인합니다.
metadatas["source"][:5]

['SPRi AI Brief_2024년1월호_F.pdf',
 'SPRi AI Brief_2024년1월호_F.pdf',
 'SPRi AI Brief_2024년1월호_F.pdf',
 'SPRi AI Brief_2024년1월호_F.pdf',
 'SPRi AI Brief_2024년1월호_F.pdf']

In [89]:
# 문서 개수 확인, 소스 개수 확인, 페이지 개수 확인
len(contents), len(metadatas["source"]), len(metadatas["page"])

(866, 866, 866)

### API 키 발급

- [링크](https://app.pinecone.io/)
- 프로필 - Account - Projects - Starter - API keys - 발급

`.env` 파일에 아래와 같이 추가합니다.

```
PINECONE_API_KEY="YOUR_PINECONE_API_KEY"
```

## 새로운 VectorStore 인덱스 생성

Pinecone 의 새로운 인덱스를 생성합니다.

![pinecone-01.png](./images/pinecone-01.png)

pinecone 인덱스를 생성합니다.

**주의사항**
- `metric` 은 유사도 측정 방법을 지정합니다. 만약 HybridSearch 를 고려하고 있다면 `metric` 은 `dotproduct` 로 지정합니다.

In [90]:
import os
from langchain_teddynote.community.pinecone import create_index

# Pinecone 인덱스 생성
pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name="teddynote-db-index",  # 인덱스 이름을 지정합니다.
    dimension=4096,  # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
    metric="dotproduct",  # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine)
)

[create_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0},
                'teddynote-namespace-01': {'vector_count': 867},
                'teddynote-namespace-02': {'vector_count': 867}},
 'total_vector_count': 1734}


아래는 **유료 Pod** 를 사용하는 예시입니다. **유료 Pod** 는 무료 Serverless Pod 대비 더 확장된 기능을 제공합니다.

- 참고: https://docs.pinecone.io/guides/indexes/choose-a-pod-type-and-size

In [91]:
import os
from langchain_teddynote.community.pinecone import create_index
from pinecone import PodSpec

# Pinecone 인덱스 생성
pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name="teddynote-db-index2",  # 인덱스 이름을 지정합니다.
    dimension=4096,  # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
    metric="dotproduct",  # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine)
    pod_spec=PodSpec(
        environment="us-west1-gcp", pod_type="p1.x1", pods=1
    ),  # 유료 Pod 사용
)

[create_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


## Sparse Encoder 생성

- Sparse Encoder 를 생성합니다. 
- `Kiwi Tokenizer` 와 한글 불용어(stopwords) 처리를 수행합니다.
- Sparse Encoder 를 활용하여 contents 를 학습합니다. 여기서 학습한 인코드는 VectorStore 에 문서를 저장할 때 Sparse Vector 를 생성할 때 활용합니다.

In [92]:
from langchain_teddynote.community.pinecone import (
    create_sparse_encoder,
    fit_sparse_encoder,
)

# 한글 불용어 사전 + Kiwi 형태소 분석기를 사용합니다.
sparse_encoder = create_sparse_encoder(stopwords(), mode="kiwi")

Sparse Encoder 에 Corpus 를 학습합니다.

- `save_path`: Sparse Encoder 를 저장할 경로입니다. 추후에 `pickle` 형식으로 저장한 Sparse Encoder 를 불러와 Query 임베딩할 때 사용합니다. 따라서, 이를 저장할 경로를 지정합니다.

In [94]:
# Sparse Encoder 를 사용하여 contents 를 학습
saved_path = fit_sparse_encoder(
    sparse_encoder=sparse_encoder, contents=contents, save_path="./sparse_encoder.pkl"
)

  0%|          | 0/866 [00:00<?, ?it/s]

[fit_sparse_encoder]
Saved Sparse Encoder to: ./sparse_encoder.pkl


[선택사항] 아래는 나중에 학습하고 저장한 Sparse Encoder 를 다시 불러와야 할 때 사용하는 코드입니다.    

In [95]:
from langchain_teddynote.community.pinecone import load_sparse_encoder

# 추후에 학습된 sparse encoder 를 불러올 때 사용합니다.
sparse_encoder = load_sparse_encoder("./sparse_encoder.pkl")

[load_sparse_encoder]
Loaded Sparse Encoder from: ./sparse_encoder.pkl


### Pinecone: DB Index에 추가 (Upsert)

![](./images/pinecone-02.png)

- `context`: 문서의 내용입니다.
- `page`: 문서의 페이지 번호입니다.
- `source`: 문서의 출처입니다.
- `values`: Embedder 를 통해 얻은 문서의 임베딩입니다.
- `sparse values`: Sparse Encoder 를 통해 얻은 문서의 임베딩입니다.

In [96]:
from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings

openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
upstage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")

분산 처리를 하지 않고 배치 단위로 문서를 Upsert 합니다. 문서의 양이 많지 않다면 아래의 방식을 사용하세요.

In [97]:
%%time
from langchain_teddynote.community.pinecone import upsert_documents
from langchain_upstage import UpstageEmbeddings

upsert_documents(
    index=pc_index,  # Pinecone 인덱스
    namespace="teddynote-namespace-01",  # Pinecone namespace
    contents=contents,  # 이전에 전처리한 문서 내용
    metadatas=metadatas,  # 이전에 전처리한 문서 메타데이터
    sparse_encoder=sparse_encoder,  # Sparse encoder
    embedder=upstage_embeddings,
    batch_size=32,
)

  0%|          | 0/28 [00:00<?, ?it/s]

[upsert_documents]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 866}},
 'total_vector_count': 866}
CPU times: user 14.4 s, sys: 576 ms, total: 14.9 s
Wall time: 2min 37s


아래는 분산처리를 수행하여 대용량 문서를 빠르게 Upsert 합니다. 대용량 업로드시 활용하세요.

In [98]:
%%time
from langchain_teddynote.community.pinecone import upsert_documents_parallel

upsert_documents_parallel(
    index=pc_index,  # Pinecone 인덱스
    namespace="teddynote-namespace-02",  # Pinecone namespace
    contents=contents,  # 이전에 전처리한 문서 내용
    metadatas=metadatas,  # 이전에 전처리한 문서 메타데이터
    sparse_encoder=sparse_encoder,  # Sparse encoder
    embedder=upstage_embeddings,
    batch_size=64,
    max_workers=30,
)

문서 Upsert 중:   0%|          | 0/14 [00:00<?, ?it/s]

총 866개의 Vector 가 Upsert 되었습니다.
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 866},
                'teddynote-namespace-02': {'vector_count': 866}},
 'total_vector_count': 1732}
CPU times: user 11.3 s, sys: 306 ms, total: 11.6 s
Wall time: 14.8 s


## 인덱스 조회/삭제

`describe_index_stats` 메서드는 인덱스의 내용에 대한 통계 정보를 제공합니다. 이 메서드를 통해 네임스페이스별 벡터 수와 차원 수 등의 정보를 얻을 수 있습니다.

**매개변수**
* `filter` (Optional[Dict[str, Union[str, float, int, bool, List, dict]]]): 특정 조건에 맞는 벡터들에 대한 통계만 반환하도록 하는 필터. 기본값은 None
* `**kwargs`: 추가 키워드 인자

**반환값**
* `DescribeIndexStatsResponse`: 인덱스에 대한 통계 정보를 담고 있는 객체

**사용 예시**
* 기본 사용: `index.describe_index_stats()`
* 필터 적용: `index.describe_index_stats(filter={'key': 'value'})`

**참고**
- metadata 필터링은 유료 사용자에 한하여 가능합니다.

In [99]:
# 인덱스 조회
pc_index.describe_index_stats()

{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 866},
                'teddynote-namespace-02': {'vector_count': 866}},
 'total_vector_count': 1732}

### 네임스페이스(namespace) 삭제

In [100]:
from langchain_teddynote.community.pinecone import delete_namespace

delete_namespace(
    pinecone_index=pc_index,
    namespace="teddynote-namespace-01",
)

네임스페이스 'teddynote-namespace-01'의 모든 데이터가 삭제되었습니다.


In [101]:
pc_index.describe_index_stats()

{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-02': {'vector_count': 866}},
 'total_vector_count': 866}

아래는 유료 사용자 전용 기능입니다. 유료 사용자는 metadata 필터링을 사용할 수 있습니다.

In [102]:
from langchain_teddynote.community.pinecone import delete_by_filter

# metadata 필터링(유료 기능) 으로 삭제
delete_by_filter(
    pinecone_index=pc_index,
    namespace="teddynote-namespace-02",
    filter={"source": {"$eq": "SPRi AI Brief_8월호_산업동향.pdf"}},
)
pc_index.describe_index_stats()

{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-02': {'vector_count': 743}},
 'total_vector_count': 743}

## 검색기(Retriever) 생성

### PineconeKiwiHybridRetriever 초기화 파라미터 설정

`init_pinecone_index` 함수와 `PineconeKiwiHybridRetriever` 클래스는 Pinecone을 사용한 하이브리드 검색 시스템을 구현합니다. 이 시스템은 밀집 벡터와 희소 벡터를 결합하여 효과적인 문서 검색을 수행합니다.

**Pinecone 인덱스 초기화**

`init_pinecone_index` 함수는 Pinecone 인덱스를 초기화하고 필요한 구성 요소를 설정합니다.

**매개변수**
* `index_name` (str): Pinecone 인덱스 이름
* `namespace` (str): 사용할 네임스페이스
* `api_key` (str): Pinecone API 키
* `sparse_encoder_pkl_path` (str): 희소 인코더 피클 파일 경로
* `stopwords` (List[str]): 불용어 리스트
* `tokenizer` (str): 사용할 토크나이저 (기본값: "kiwi")
* `embeddings` (Embeddings): 임베딩 모델
* `top_k` (int): 반환할 최대 문서 수 (기본값: 10)
* `alpha` (float): 밀집 벡터와 희소 벡터의 가중치 조절 파라미터 (기본값: 0.5)

**주요 기능**
1. Pinecone 인덱스 초기화 및 통계 정보 출력
2. 희소 인코더(BM25) 로딩 및 토크나이저 설정
3. 네임스페이스 지정

In [103]:
from langchain_teddynote.community.pinecone import init_pinecone_index

pinecone_params = init_pinecone_index(
    index_name="teddynote-db-index2",  # Pinecone 인덱스 이름
    namespace="teddynote-namespace-02",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_path="./sparse_encoder.pkl",  # Sparse Encoder 저장경로(save_path)
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",
    embeddings=UpstageEmbeddings(
        model="solar-embedding-1-large-query"
    ),  # Dense Embedder
    top_k=5,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75로 설정한 경우, (0.75: Dense Embedding, 0.25: Sparse Embedding)
)

[init_pinecone_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-02': {'vector_count': 743}},
 'total_vector_count': 743}


### PineconeKiwiHybridRetriever

`PineconeKiwiHybridRetriever` 클래스는 Pinecone과 Kiwi를 결합한 하이브리드 검색기를 구현합니다.

**주요 속성**
* `embeddings`: 밀집 벡터 변환용 임베딩 모델
* `sparse_encoder`: 희소 벡터 변환용 인코더
* `index`: Pinecone 인덱스 객체
* `top_k`: 반환할 최대 문서 수
* `alpha`: 밀집 벡터와 희소 벡터의 가중치 조절 파라미터
* `namespace`: Pinecone 인덱스 내 네임스페이스

**특징**
* 밀집 벡터와 희소 벡터를 결합한 HybridSearch Retriever
* 가중치 조절을 통한 검색 전략 최적화 가능
* 다양한 동적 metadata 필터링 적용 가능(`search_kwargs` 사용: `filter`, `k`, `rerank`, `rerank_model`, `top_n` 등)

**사용 예시**
1. `init_pinecone_index` 함수로 필요한 구성 요소 초기화
2. 초기화된 구성 요소로 `PineconeKiwiHybridRetriever` 인스턴스 생성
3. 생성된 검색기를 사용하여 하이브리드 검색 수행

`PineconeKiwiHybridRetriever` 를 생성합니다.

In [104]:
from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetriever

# 검색기 생성
pinecone_retriever = PineconeKiwiHybridRetriever(**pinecone_params)

일반 검색

In [105]:
# 실행 결과
search_results = pinecone_retriever.invoke("gpt-4o 미니 출시 관련 정보에 대해서 알려줘")
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
7
오픈AI, 사람과 자연스러운 실시간 대화가 가능한 ‘GPT-4o’ 출시
n 오픈AI가 응답시간이 최소 0.23초, 평균 0.32초에 불과해 사람과 실시간 음성 대화가 가능한 
신규 AI 모델 ‘GPT-4o’를 무료로 공개
n GPT-4o는 API로 제공 시 기존 GPT 모델보다 처리속도가 2배 빠르고 비용은 절반 수준이며, 
다국어, 오디오, 이미지 관련 벤치마크 테스트에서 최고 수준의 성능을 기록
KEY Contents
{'page': 9.0, 'source': 'SPRi AI Brief_6월호_산업동향 최종.pdf', 'score': 0.3255687}


GPT-4 터보의 86.5%와 구글 제미나이 울트라의 83.7%를 넘어섰음
n GPT-4o는 전 세계 사용자들에게 무료로 제공되며, 유료 사용자는 무료 사용자보다 5배 더 많은 
질문을 할 수 있음 
∙GPT-4o의 텍스트와 이미지 기능은 5월 13일부터 바로 제공되며, 유료 서비스인 챗GPT 플러스 
사용자에게는 음성이 지원되는 신규 버전이 수 주 안에 출시될 예정
∙개발자들은 API로 GPT-4o의 텍스트와 이미지 기능을 사용할 수 있으며, 오디오와 비디오 기능은 수 
주 안에 일부 파트너 집단에 선공개할 예정
{'page': 9.0, 'source': 'SPRi AI Brief_6월호_산업동향 최종.pdf', 'score': 0.3008562}


2. 기업/산업 
   ▹ 오픈AI, 사람과 자연스러운 실시간 대화가 가능한 ‘GPT-4o’ 출시··································· 7
   ▹ AI 안전 연구를 맡은 오픈AI 슈퍼얼라인먼트팀, 핵심 인력 퇴진과 함께 해체·················· 8
   ▹ 구글, 연례 개발자회의에서 ‘제미나이’ 신모델과 에이전트 등 생성 AI 제품 대거 발표··· 9
{'page': 1.0, 'source'

동적 `search_kwargs` 사용
- `k`: 반환할 최대 문서 수 지정

In [106]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "gpt-4o 미니 출시 관련 정보에 대해서 알려줘", search_kwargs={"k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
7
오픈AI, 사람과 자연스러운 실시간 대화가 가능한 ‘GPT-4o’ 출시
n 오픈AI가 응답시간이 최소 0.23초, 평균 0.32초에 불과해 사람과 실시간 음성 대화가 가능한 
신규 AI 모델 ‘GPT-4o’를 무료로 공개
n GPT-4o는 API로 제공 시 기존 GPT 모델보다 처리속도가 2배 빠르고 비용은 절반 수준이며, 
다국어, 오디오, 이미지 관련 벤치마크 테스트에서 최고 수준의 성능을 기록
KEY Contents
{'page': 9.0, 'source': 'SPRi AI Brief_6월호_산업동향 최종.pdf', 'score': 0.3255687}




동적 `search_kwargs` 사용
- `alpha`: 밀집 벡터와 희소 벡터의 가중치 조절 파라미터. 0과 1 사이의 값을 지정합니다. `0.5` 가 기본값이고, 1에 가까울수록 dense 벡터의 가중치가 높아집니다.

In [112]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽", search_kwargs={"alpha": 1, "k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

n 앤스로픽은 개발자 콘솔도 업데이트하여, 개발자가 여러 프롬프트를 생성하고 테스트함으로써 
클로드를 최적화할 수 있도록 지원
∙앤스로픽은 사용자가 클로드에 맞춤형 지침을 제공할 수 있는 ‘시스템 프롬프트’ 기능도 도입할 
예정으로, 이 기능은 클로드가 사용자 요구에 맞춤화된 방식으로 체계적 응답을 할 수 있도록 지원
☞ 출처 : Anthropic, Introducing Claude 2.1, 2023.11.21.
{'page': 7.0, 'source': 'SPRi AI Brief_2024년1월호_F.pdf', 'score': 0.3507563}




In [113]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽", search_kwargs={"alpha": 0, "k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
11
앤스로픽 연구 결과, AI도 사람처럼 의도적으로 거짓말 가능
n 앤스로픽이 특정 프롬프트가 주어지면 사용자를 속이고 악성코드를 출력하는 등, 악의적 
행동을 할 수 있는 LLM을 연구
n 앤스로픽에 따르면 기만적 행동을 하는 LLM에 대한 안전성 훈련 이후에도 이를 제거할 수 
없었으며 오히려 해당 행동을 유발하는 프롬프트에 대한 반응의 정확도가 향상됨 
n
KEY Contents
£ 앤스로픽, 사용자를 속이고 악성코드를 출력하는 LLM 연구
{'page': 13.0, 'source': 'SPRi AI Brief_2024년3월호_F.pdf', 'score': 0.7349932}




**Metadata 필터링**

![](./images/pinecone-metadata.png)

동적 `search_kwargs` 사용
- `filter`: metadata 필터링 적용

(예시) `page` 가 5보다 작은 문서만 검색합니다.

In [116]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 claude 출시 관련 내용을 알려줘",
    search_kwargs={"filter": {"page": {"$lt": 5}}, "k": 2},
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

▹ 앤스로픽, 최신 AI 모델 ‘클로드 3.5 소네트’ 출시··························································· 10
   ▹ 프랑스의 미스트랄AI, 6억 유로 규모의 투자 유치  ························································ 11
{'page': 1.0, 'source': 'SPRi AI Brief_7월호_산업동향.pdf', 'score': 0.31394503}


2. 기업/산업 
   ▹ 구글, 멀티모달 AI 모델 ‘제미나이’ 공개············································································  3
   ▹ 구글 클라우드, 기업용 AI 플랫폼에 이미지 생성 AI ‘이매진 2’ 추가  ·························  4
   ▹ 앤스로픽, 20만 개의 토큰을 입력할 수 있는 ‘클로드 2.1’ 공개 ···································  5
{'page': 1.0, 'source': 'SPRi AI Brief_2024년1월호_F.pdf', 'score': 0.2237935}




동적 `search_kwargs` 사용
- `filter`: metadata 필터링 적용

(예시) `source` 가 `SPRi AI Brief_8월호_산업동향.pdf` 문서내 검색합니다.

In [117]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 claude 3.5 출시 관련 내용을 알려줘",
    search_kwargs={
        "filter": {"source": {"$eq": "SPRi AI Brief_7월호_산업동향.pdf"}},
        "k": 3,
    },
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

▹ 앤스로픽, 최신 AI 모델 ‘클로드 3.5 소네트’ 출시··························································· 10
   ▹ 프랑스의 미스트랄AI, 6억 유로 규모의 투자 유치  ························································ 11
{'page': 1.0, 'source': 'SPRi AI Brief_7월호_산업동향.pdf', 'score': 0.357203}


1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
9
앤스로픽, 최신 AI 모델 ‘클로드 3.5 소네트’ 출시 
n 앤스로픽이 최신 AI 모델 ‘클로드 3.5 소네트’을 공개하고 이전 모델 중 가장 고성능 모델인 
‘클로드 3 오퍼스’나 오픈AI의 최신 모델 ‘GPT-4o’보다 성능이 뛰어나다고 강조
n 앤스로픽은 클로드로 생성한 코딩이나 문서 등을 실시간으로 확인해 편집과 같은 작업을 할 수 
있는 ‘아티팩트’ 신기능도 출시
KEY Contents
{'page': 11.0, 'source': 'SPRi AI Brief_7월호_산업동향.pdf', 'score': 0.35153997}


있는 ‘아티팩트’ 신기능도 출시
KEY Contents
£ 클로드 3.5 소네트, 이전 대표모델 클로드 3 오퍼스와 오픈AI의 GPT-4o 능가
n 앤스로픽이 2024년 6월 21일 ‘클로드(Claude) 3.5’ 모델군 중 첫 번째로 ‘소네트(Sonnet)’를 공개
했으며, 하반기에 경량 모델 ‘하이쿠(Haiku)’와 가장 강력한 ‘오퍼스(Opus)’를 출시할 계획
∙소네트는 클로드 웹사이트(Claude.ai) 및 iOS 앱에서 무료로 제공되며, 유료 가입자에게는 이용 
한도를 대폭 넓혀서 제공
{'page': 11.0, 'source': 'SPRi AI Brief_7월호_산업동향.pdf', 'score': 0.34043252}

### Reranking 적용

아직은 `pre` 기능입니다.

- 동적 reranking 기능을 구현해 놓았지만, pinecone 라이브러리 의존성에 문제가 있을 수 있습니다.
- 따라서, 아래 코드는 향후 의존성 해결 후 원활하게 동작할 수 있습니다.

참고 문서: https://docs.pinecone.io/guides/inference/rerank

In [ ]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 클로드 소넷",
    search_kwargs={"rerank": True, "rerank_model": "bge-reranker-v2-m3", "top_n": 3},
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")